In [ ]:
from dolfin import *
from mshr import *
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
N = 20 # resolution
R = 0.1  # brain circle Radius
ventricle_radius = R/3
aqueduct_width = R/5

brain = Circle(Point(0,0), R)
ventricle = Circle(Point(0,0), ventricle_radius)
aqueduct = Rectangle(Point(-aqueduct_width/2, -R), Point(aqueduct_width/2, 0))

brain = brain - ventricle - aqueduct
mesh = generate_mesh(brain, N)


In [ ]:
def solve_darcy(mesh, K):
    V = FunctionSpace(mesh, "CG", 1)

    u = Function(V)
    K = Constant(K)
    f = Constant(0.0)
    v = TestFunction(V)
    mmHg2Pa = 133.32
    a_cardiac = 1.5 * 7
    L = R - ventricle_radius

    F = ( inner(K*grad(u), grad(v)) - f * v) * dx
    
    ventricle = CompiledSubDomain("on_boundary && (x[0]*x[0] + x[1]*x[1] < R*R)", R = R)
    skull = CompiledSubDomain("on_boundary && (x[0]*x[0] + x[1]*x[1] >= R*R*0.99 )", R = R)

    bc_ventricle = DirichletBC(V, 0.0, ventricle, check_midpoint=False)
    bc_skull = DirichletBC(V, mmHg2Pa*a_cardiac*L, skull, check_midpoint=False)

    bcs = [bc_ventricle, bc_skull]

    solve(F==0, u, bcs)
    return u


In [ ]:
k_values = [1e-2, 1e-3, 1e-4]
x_vals = np.linspace(0, R, 100)
y = 0
y_vals = np.repeat(y, 100)
in_domain = x_vals**2 + y_vals**2 > ventricle_radius**2

x_in_domain = x_vals[in_domain]
plt.figure(figsize=(15,8))


for i, k in enumerate(k_values):
    plt.subplot(2, len(k_values), i + 1)
    u = solve_darcy(mesh, k)
    plot(u)
    plt.title("$K=${0}".format(k))
    plt.subplot(2, len(k_values), i + len(k_values) + 1)
    p_vals = [u(Point(x,y)) for x in x_in_domain]
    plt.plot(x_in_domain, p_vals, "*")
    plt.grid()
    plt.ylabel("p in Pa")
    plt.xlabel("x in m")
    plt.title("$K=${0}".format(k))